# Modbus TCP Register Reader - Jupyter Notebook Examples

This notebook demonstrates how to read Modbus TCP holding registers with various data types.


## 1. Install Required Packages

Run this cell first to install pymodbus if not already installed.


In [ ]:
# Install required packages (run once)
# !pip install pymodbus pandas


## 2. Import the Modbus Reader


In [ ]:
from modbus_reader import ModbusReader
import pandas as pd
from datetime import datetime
import time


## 3. Initialize and Connect


In [ ]:
# Create Modbus reader instance
reader = ModbusReader(
    ip_address="192.168.0.5",
    port=502,
    slave_id=255
)

# Connect to device
reader.connect()


## 4. Read Different Data Types

### Read INT16 (16-bit integer)


In [ ]:
# Read single INT16 value at address 0
value = reader.read_int16(address=0, count=1, signed=True)
print(f"INT16 at address 0: {value}")

# Read multiple INT16 values
values = reader.read_int16(address=0, count=5, signed=True)
print(f"INT16 values from address 0 (5 values): {values}")


### Read INT32 (32-bit integer)


In [ ]:
# Read INT32 at address 10 (uses 2 registers: 10 and 11)
value = reader.read_int32(address=10, count=1, byte_order="big")
print(f"INT32 at address 10: {value}")


### Read FLOAT32 (32-bit floating point)


In [ ]:
# Read FLOAT32 at address 20 (uses 2 registers: 20 and 21)
value = reader.read_float32(address=20, count=1, byte_order="big")
print(f"FLOAT32 at address 20: {value}")


### Read FLOAT64 (64-bit floating point)


In [ ]:
# Read FLOAT64 at address 30 (uses 4 registers: 30, 31, 32, 33)
value = reader.read_float64(address=30, count=1, byte_order="big")
print(f"FLOAT64 at address 30: {value}")


### Read BITMAP (bit field)


In [ ]:
# Read bitmap from address 40 (16 bits from 1 register)
bits = reader.read_bitmap(address=40, count=1)
print(f"Bitmap at address 40: {bits}")
print(f"Bit 0: {bits[0]}, Bit 1: {bits[1]}, ..., Bit 15: {bits[15]}")


## 5. Read Multiple Registers with Register Map


In [ ]:
# Define your register map
register_map = [
    {
        'name': 'VR',
        'address': 0,
        'data_type': 'FLOAT32',
        'count': 1,
        'unit': 'V',
        'description': 'Voltage Phase R',
        'options': {'byte_order': 'big'}
    },
    {
        'name': 'VY',
        'address': 2,
        'data_type': 'FLOAT32',
        'count': 1,
        'unit': 'V',
        'description': 'Voltage Phase Y',
        'options': {'byte_order': 'big'}
    },
    {
        'name': 'VB',
        'address': 4,
        'data_type': 'FLOAT32',
        'count': 1,
        'unit': 'V',
        'description': 'Voltage Phase B',
        'options': {'byte_order': 'big'}
    },
    {
        'name': 'IR',
        'address': 10,
        'data_type': 'FLOAT32',
        'count': 1,
        'unit': 'A',
        'description': 'Current Phase R',
        'options': {'byte_order': 'big'}
    },
    {
        'name': 'Ptotal',
        'address': 30,
        'data_type': 'FLOAT64',
        'count': 1,
        'unit': 'kW',
        'description': 'Total Power',
        'options': {'byte_order': 'big'}
    },
    {
        'name': 'Status',
        'address': 50,
        'data_type': 'INT16',
        'count': 1,
        'unit': '',
        'description': 'Device Status',
        'options': {'signed': True}
    },
    {
        'name': 'Flags',
        'address': 51,
        'data_type': 'BITMAP',
        'count': 1,
        'unit': '',
        'description': 'Status Flags'
    }
]

# Read all registers and get DataFrame
df = reader.read_multiple(register_map)
df


## 6. Continuous Reading (Polling)


In [ ]:
# Read values every 5 seconds
for i in range(10):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Read a few key parameters
    vr = reader.read_float32(0)
    vy = reader.read_float32(2)
    vb = reader.read_float32(4)
    ptotal = reader.read_float64(30)
    
    print(f"[{timestamp}] VR={vr}V, VY={vy}V, VB={vb}V, Ptotal={ptotal}kW")
    
    time.sleep(5)  # Wait 5 seconds


## 7. Disconnect


In [ ]:
# Always disconnect when done
reader.disconnect()
